In [11]:
import pandas as pd
import numpy as np
import json
import re

pd.set_option('display.max_columns', None)

In [12]:
def createExpansionDictionnary(data):
    expansionDict = {}

    for expansion in data['card_sets']:
        if expansion['type'] != 'EXPANSION':
            continue

        expansionDict[expansion['id']] = expansion['name']

    return expansionDict

def createName(name, subtitle):
    return name.capitalize()  + ' - ' + subtitle if subtitle != '' else name.capitalize()

In [13]:
f = open('data/cards.json')

# returns JSON object as a dictionary
data = json.load(f)
df = pd.DataFrame()

# Iterating through the json list
for cardType in data['cards']:
    for card in data['cards'][cardType]:
        newDf = pd.json_normalize(card)
        newDf['Type'] = cardType
        df = pd.concat([df, newDf])

# Closing file
f.close()

df

,name,subtitle,strength,willpower,quest_value,rarity,ink_cost,author,deck_building_id,culture_invariant_id,sort_number,additional_info,ink_convertible,abilities,subtypes,flavor_text,rules_text,card_identifier,image_urls,foil_mask_url,card_sets,magic_ink_colors,searchable_keywords,Type,foil_type,special_rarity_id,deck_building_limit,varnish_type,varnish_mask_url,foil_detail_url,move_cost
0,ARIEL,Sur des jambes humaines,3.0,4.0,2.0,UNCOMMON,4,Matthew Robert Davies,086f72176bc2fbb2a19898745a8218e1fe826c00,1,1,[],True,[],"[Storyborn, Héros, Princesse]",…,SANS VOIX Ce personnage ne peut pas être pour ...,1/204 FR 1,"[{'height': 2048, 'url': 'https://api.lorcana....",https://api.lorcana.ravensburger.com/images/fr...,[1],[AMBER],"[Little Mermaid, La Petite Sirène]",characters,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,ARIEL,Chanteuse exceptionnelle,2.0,3.0,1.0,SUPER,3,Alice Pisoni,cae9d71be6c7f2ae989356aff5c1d3e307890630,2,2,[],True,[Mélomane],"[Storyborn, Héros, Princesse]",,Mélomane 5 (Ce personnage est considéré comme ...,2/204 FR 1,"[{'height': 2048, 'url': 'https://api.lorcana....",https://api.lorcana.ravensburger.com/images/fr...,[1],[AMBER],"[Little Mermaid, La Petite Sirène]",characters,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,CENDRILLON,Douce et charmante,2.0,5.0,2.0,UNCOMMON,4,Javier Salas,933453208f4cab4b5eacd5391db6a6cf95b7e691,3,3,[],True,[Mélomane],"[Storyborn, Héros, Princesse]",,Mélomane 5 (Ce personnage est considéré comme ...,3/204 FR 1,"[{'height': 2048, 'url': 'https://api.lorcana....",https://api.lorcana.ravensburger.com/images/fr...,[1],[AMBER],"[Cendrillon, Cinderella]",characters,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,DINGO,Mousquetaire,3.0,6.0,1.0,UNCOMMON,5,Jochem Van Gool,88974b7ccdf603a29b402df56365c9ac1c82289f,4,4,[],True,[Rempart],"[Dreamborn, Héros, Mousquetaire]","En garde, hyuck!",Rempart (Ce personnage peut entrer en jeu épui...,4/204 FR 1,"[{'height': 2048, 'url': 'https://api.lorcana....",https://api.lorcana.ravensburger.com/images/fr...,[1],[AMBER],NaN,characters,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,HADÈS,Roi de l'Olympe,6.0,7.0,1.0,RARE,8,Alex Accorsi,b49576fe526d49f6abcdf5af9e3eb03f64505194,5,5,[],False,[Alter],"[Floodborn, Méchant, Roi, Dieu]","Oh, mais j'y pense, il va me falloir de nouvel...",Alter 6 (Vous pouvez payer 6 pour jouer ce per...,5/204 FR 1,"[{'height': 2048, 'url': 'https://api.lorcana....",https://api.lorcana.ravensburger.com/images/fr...,[1],[AMBER],"[Hercules, Hercule]",characters,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Tempête de sable désastreuse,NaN,NaN,NaN,NaN,SPECIAL,4,Pietro B. Zemelo / Theonidas,f3a6d5926f5a0a5636c5fcbcb6aa847824246c8a,1915,1915,[],True,[],[],Vous devrez creuser en profondeur si vous voul...,Chaque Illumineur choisit l'un de ses objets o...,20/35 FR Q2,"[{'height': 2048, 'url': 'https://api.lorcana....",NaN,[10],[],[Aladdin],actions,None,5.0,NaN,NaN,NaN,NaN,NaN
0,Puissance soudaine,NaN,NaN,NaN,NaN,SPECIAL,3,Gonzalo Kenny,03287ba12c26ceb166cadd2e9812e02513dfb6e7,1916,1916,[],True,[],[],Ah ! Je le sens ! Tout Lorcana tremblera devan...,"Jafar gagne 2 éclats de Lore. Ensuite, placez ...",21/35 FR Q2,"[{'height': 2048, 'url': 'https://api.lorcana....",NaN,[10],[],[Aladdin],actions,None,5.0,NaN,NaN,NaN,NaN,NaN
0,Séisme arcanique,NaN,NaN,NaN,NaN,SPECIAL,4,French Carlomagno,46e5e916f27135819de17ac3aed108aff3b64569,1917,1917,[],True,[],[],Votre plan est en train de s'effondrer. Tentez...,Jafar épuise tous les personnages adverses. Il...,22/35 FR Q2,"[{'height': 2048, 'url': 'https://api.lorcana....",NaN,[10],[],[Aladdin],actions,None,5.0,NaN,NaN,NaN,NaN,NaN
0,Cyclone cataclysmique,NaN,NaN,NaN,NaN,SPECIAL,6,Douglas De La Hoz,f271b39d14fabd7d429ab5e196ab1470fbb5c31e,1918,1918,[],True,[],[],Voilà de quoi vous décoiffer !%—Jafar,Chaque Illumineur choisit l'un de ses personna...,23/35 FR Q2,"[{'height': 2048, 'url': 'https://api.lorcana....",NaN,[10],[],[Lorcana],actions,None,5.0,NaN,NaN,N

In [14]:
expansionDict = createExpansionDictionnary(data)

In [15]:
df['subtitle'] = df['subtitle'].fillna('')

df['card_sets'] = df['card_sets'].apply(lambda x : x[0])
df = df.loc[df.card_sets.isin(list(expansionDict.keys()))]
df['card_sets'].value_counts()

card_sets
1    239
3    234
4    233
5    233
6    233
8    233
7    232
2    227
Name: count, dtype: int64

In [16]:
df['Name'] = df.apply(lambda x: createName(x['name'], x['subtitle']), axis=1)
df['SetName'] = df['card_sets'].apply(lambda x : expansionDict[x])
df['magic_ink_colors'] = df['magic_ink_colors'].apply(lambda x : re.sub(r"\[|\]|'", "", str(x)).capitalize())
df['subtypes'] = df['subtypes'].apply(lambda x : re.sub(r"\[|\]|'", "", str(x)))
df['abilities'] = df['abilities'].apply(lambda x : re.sub(r"\[|\]|'", "", str(x)))
df['rarity'] = df['rarity'].apply(lambda x : x.capitalize())
df['CardNumber'] = df['card_identifier'].apply(lambda x : re.findall(r"^\d+", str(x))[0])
df['Image'] = df['image_urls'].apply(lambda x : re.sub(r"\?checksum.*", "", x[1]['url']))
df['OwnedNumber'] = 0
df = df.drop(columns=['name', 'subtitle', 'deck_building_id', 'culture_invariant_id', 'sort_number', 'additional_info', 'card_identifier', 'image_urls', 'foil_mask_url', 'foil_type', 'special_rarity_id', 'deck_building_limit', 'varnish_type', 'varnish_mask_url'])
df

C:\Users\flore\AppData\Local\Temp\ipykernel_17024\1503139365.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Name'] = df.apply(lambda x: createName(x['name'], x['subtitle']), axis=1)
C:\Users\flore\AppData\Local\Temp\ipykernel_17024\1503139365.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SetName'] = df['card_sets'].apply(lambda x : expansionDict[x])
C:\Users\flore\AppData\Local\Temp\ipykernel_17024\1503139365.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

,strength,willpower,quest_value,rarity,ink_cost,author,ink_convertible,abilities,subtypes,flavor_text,rules_text,card_sets,magic_ink_colors,searchable_keywords,Type,foil_detail_url,move_cost,Name,SetName,CardNumber,Image,OwnedNumber
0,3.0,4.0,2.0,Uncommon,4,Matthew Robert Davies,True,,"Storyborn, Héros, Princesse",…,SANS VOIX Ce personnage ne peut pas être pour ...,1,Amber,"[Little Mermaid, La Petite Sirène]",characters,NaN,NaN,Ariel - Sur des jambes humaines,Premier Chapitre,1,https://api.lorcana.ravensburger.com/images/fr...,0
0,2.0,3.0,1.0,Super,3,Alice Pisoni,True,Mélomane,"Storyborn, Héros, Princesse",,Mélomane 5 (Ce personnage est considéré comme ...,1,Amber,"[Little Mermaid, La Petite Sirène]",characters,NaN,NaN,Ariel - Chanteuse exceptionnelle,Premier Chapitre,2,https://api.lorcana.ravensburger.com/images/fr...,0
0,2.0,5.0,2.0,Uncommon,4,Javier Salas,True,Mélomane,"Storyborn, Héros, Princesse",,Mélomane 5 (Ce personnage est considéré comme ...,1,Amber,"[Cendrillon, Cinderella]",characters,NaN,NaN,Cendrillon - Douce et charmante,Premier Chapitre,3,https://api.lorcana.ravensburger.com/images/fr...,0
0,3.0,6.0,1.0,Uncommon,5,Jochem Van Gool,True,Rempart,"Dreamborn, Héros, Mousquetaire","En garde, hyuck!",Rempart (Ce personnage peut entrer en jeu épui...,1,Amber,NaN,characters,NaN,NaN,Dingo - Mousquetaire,Premier Chapitre,4,https://api.lorcana.ravensburger.com/images/fr...,0
0,6.0,7.0,1.0,Rare,8,Alex Accorsi,False,Alter,"Floodborn, Méchant, Roi, Dieu","Oh, mais j'y pense, il va me falloir de nouvel...",Alter 6 (Vous pouvez payer 6 pour jouer ce per...,1,Amber,"[Hercules, Hercule]",characters,NaN,NaN,Hadès - Roi de l'Olympe,Premier Chapitre,5,https://api.lorcana.ravensburger.com/images/fr...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,NaN,NaN,NaN,Enchanted,9,Samanta Erdini,False,"""À lunisson""",Chanson,,À l'unisson 9 (Vous pouvez n'importe quel nomb...,8,Amethyst,"[Encanto, Encanto : La Fantastique Famille Mad...",actions,NaN,NaN,Miraculeux et merveilleux,Le Règne de Jafar,212,https://api.lorcana.ravensburger.com/images/fr...,0
0,NaN,NaN,NaN,Enchanted,3,Yrgane Ramon,True,,Chanson,,(Vous pouvez un personnage coûtant 3 ou plus p...,8,"Amethyst, sapphire","[La Reine des Neiges, Frozen]",actions,NaN,NaN,Dans un autre monde,Le Règne de Jafar,213,https://api.lorcana.ravensburger.com/images/fr...,0
0,NaN,NaN,NaN,Enchanted,3,Koni,True,,,,Choisissez entre : • Choisissez un personnage ...,8,Emerald,"[Emperors New Groove, Kuzco, l'empereur Mégalo]",actions,NaN,NaN,Pas ce levier-là !,Le Règne de Jafar,216,https://api.lorcana.ravensburger.com/images/fr...,0
0,NaN,NaN,NaN,Enchanted,3,LadyShalirin,False,,,,"Si vous n'avez aucune carte en main, piochez j...",8,Steel,"[Raya et le Dernier Dragon, Raya and the Last ...",actions,NaN,NaN,Plan désespéré,Le Règne de Jafar,222,https://api.lorcana.ravensburger.com/images/fr...,0


Classifications -> Subtypes
_ -> maj
BodyText -> rulesText
Lore -> LoreValue
Set_Num -> SetNumber
Card_Num -> CardNumber

In [17]:
df = df.rename(columns={'author'            : 'Artist',
                        'card_sets'         : 'SetNumber',
                        'magic_ink_colors'  : 'Color',
                        'ink_cost'          : 'Cost',
                        'ink_convertible'   : 'Inkable',
                        'rarity'            : 'Rarity',
                        'flavor_text'       : 'FlavorText',
                        'rules_text'        : 'RulesText',
                        'quest_value'       : 'LoreValue',
                        'willpower'         : 'Willpower',
                        'strength'          : 'Strength',
                        'move_cost'         : 'MoveCost',
                        'abilities'         : 'Abilities',
                        'subtypes'          : 'Subtypes'
                        })

In [18]:
df = df.astype({'CardNumber': 'Int32',
                'Cost': 'Int32',
                'Strength': 'Int32',
                'Willpower': 'Int32',
                'LoreValue': 'Int32',
                'MoveCost': 'Int32',
                'SetNumber': 'Int32'
                })

In [19]:
newIndex = ['Name', 'CardNumber', 'Color', 'Type', 'Subtypes', 'Cost', 'Inkable', 'Rarity', 'Strength', 'Willpower', 'LoreValue', 'MoveCost', 'SetNumber', 'SetName', 'Abilities', 'RulesText', 'FlavorText', 'Artist', 'Image']
df = df[newIndex].reset_index(drop=True)

In [20]:
df = df.sort_values(by=['SetNumber', 'CardNumber'])
mask = df.Rarity == 'Special'
df = pd.concat([df.loc[~mask], df.loc[mask]], axis=0)
df.to_csv("data/cards.csv", index=False)